<a href="https://colab.research.google.com/github/atompsc/VideoGameSalesAnalysis/blob/main/Video_Game_Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
# Install dependencies as needed:
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
# Provide the correct file name from the dataset
file_path = "vgsales.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "gregorut/videogamesales",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-1-bcf53616db7c>:11: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  


In [ ]:
# แสดงข้อมูล 5 แถวแรก
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
#ตรวจสอบชนิดข้อมูล (Data Types)
df.dtypes

,0
Rank,int64
Name,object
Platform,object
Year,float64
Genre,object
Publisher,object
NA_Sales,float64
EU_Sales,float64
JP_Sales,float64
Other_Sales,float64


In [ ]:
# ตรวจสอบข้อมูลที่ขาดหาย (Missing Values)

df.isnull().sum()

,0
Rank,0
Name,0
Platform,0
Year,271
Genre,0
Publisher,58
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0


# Data Cleaning

In [ ]:
#จัดการ Missing Values
# ลบแถวที่ไม่มีข้อมูลสำคัญ เช่น Year หรือ Publisher
df_clean = df.dropna(subset=['Year', 'Publisher'])

# ตรวจสอบอีกครั้ง
df_clean.isnull().sum()


,0
Rank,0
Name,0
Platform,0
Year,0
Genre,0
Publisher,0
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0


In [ ]:
#หาก Year เป็น float แทนที่จะเป็น integer

df_clean = df.dropna(subset=['Year', 'Publisher']).copy()


In [ ]:
#ตรวจสอบชนิดข้อมูล (Data Types)
df_clean.dtypes


,0
Rank,int64
Name,object
Platform,object
Year,float64
Genre,object
Publisher,object
NA_Sales,float64
EU_Sales,float64
JP_Sales,float64
Other_Sales,float64


# Feature Engineering

In [ ]:
#สร้างฟีเจอร์ Decade (ทศวรรษของปีวางจำหน่าย)

df_clean.loc[:, 'Decade'] = (df_clean['Year'] // 10) * 10


In [ ]:
#สร้างฟีเจอร์ ยอดขายแต่ละภูมิภาคเป็น % ของ Global

region_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
for col in region_cols:
    df_clean.loc[:, f'{col}_Ratio'] = df_clean[col] / df_clean['Global_Sales']

In [ ]:
# แทนที่ 0 ด้วย pd.NA ในคอลัมน์ 'Global_Sales'
import pandas as pd
df_clean.loc[df_clean['Global_Sales'] == 0, 'Global_Sales'] = pd.NA

# ตอนนี้ลบแถวที่ 'Global_Sales' เป็น NA บนสำเนาที่สร้างขึ้นอย่างชัดเจน
df_clean.dropna(subset=['Global_Sales'], inplace=True)


In [ ]:
#สร้างฟีเจอร์ Platform Type (เช่น PlayStation, Xbox, Nintendo)

def platform_group(platform):
    if platform.startswith('PS'):
        return 'PlayStation'
    elif platform.startswith('X'):
        return 'Xbox'
    elif platform.startswith('N'):
        return 'Nintendo'
    else:
        return 'Other'

df_clean['Platform_Group'] = df_clean['Platform'].apply(platform_group)


In [ ]:
# แสดงข้อมูล 5 แถวแรก
df_clean.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Decade,NA_Sales_Ratio,EU_Sales_Ratio,JP_Sales_Ratio,Other_Sales_Ratio,Platform_Group
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2000.0,0.501450,0.350737,0.045564,0.102248,Other
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,1980.0,0.722664,0.088966,0.169235,0.019135,Nintendo
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,2000.0,0.442490,0.359576,0.105807,0.092406,Other
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2000.0,0.477273,0.333636,0.099394,0.089697,Other
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,1990.0,0.359260,0.283392,0.325789,0.031878,Other


## ขั้นตอนการเตรียมข้อมูล

### 1. Data Cleaning
- ลบแถวที่ขาดค่าของ Year และ Publisher
- แปลงชนิดข้อมูล Year เป็นจำนวนเต็ม


### 2. Feature Engineering
- สร้างฟีเจอร์ใหม่:
  - **Decade** : ใช้แบ่งช่วงปี
  - **Ratio** : สัดส่วนยอดขายแต่ละภูมิภาคเทียบกับ Global
  -  **Platform_Group** : กลุ่มของแพลตฟอร์ม เช่น PlayStation, Xbox, Nintendo


In [46]:
# บันทึก DataFrame เป็นไฟล์ CSV
# index=False คือ ไม่ต้องบันทึก index ของ DataFrame ลงในไฟล์ CSV
df_clean.to_csv('cleaned_videogamesales.csv', index=False)

# Import the files module from google.colab
from google.colab import files

# Now you can use files.download
files.download('cleaned_videogamesales.csv') # Changed the filename to match the saved file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>